<a href="https://colab.research.google.com/github/MHDBST/BERT_examples/blob/master/MaskLM_document_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf
import pandas as pd
!pip install bert-tensorflow

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

('TPU address is', 'grpc://10.26.93.186:8470')
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 12690508427458736334),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 16203115377378444841),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 17879316257458164133),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 17143706759747131924),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 3111776084791170922),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 11418182598355356184),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 16258728155362305611),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 15406828482894591040),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 10

In [0]:
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
from bert import modeling
# import optimization
# import run_classifier
from bert import run_classifier_with_tfhub
# import tokenization

# import tfhub 
import tensorflow_hub as hub

In [9]:
BUCKET = 'bert_example' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'
OUTPUT_DIR = 'gs://{}/mask_lm/models/V2/smallBERT-docLevel-seq512'.format(BUCKET)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
BERT_MODEL = 'uncased_L-12_H-768_A-12' #@param {type:"string"}
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'

***** Model output directory: gs://bert_example/mask_lm/models/V2/smallBERT-docLevel-seq512 *****


In [10]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

# BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1" ##Small Bert
# BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-24_H-1024_A-16/1" ##Big Bert
def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0429 21:43:58.652457 140603958171520 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [0]:
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 2e-3
NUM_TRAIN_EPOCHS = 5.0  ## Activate if ** is Not ACTIVATED
MAX_SEQ_LENGTH = 512
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 50
SAVE_SUMMARY_STEPS = 25




In [18]:
from tensorflow import keras
import os
import re
from sklearn.model_selection import train_test_split

# all_data = pd.read_csv('/content/mask_lm.csv', encoding='latin-1')
# all_data = all_data.sample(frac=1)


# data_train, all_data_test = train_test_split(all_data, test_size=0.2)
# data_dev,data_test =  train_test_split(all_data_test, test_size=0.5)

data_train = pd.read_csv('/content/mask_lm_3Dec_7Dec_reindex_train_v3.csv', encoding='latin-1')
data_dev = pd.read_csv('/content/mask_lm_3Dec_7Dec_reindex_dev_v3.csv', encoding='latin-1')
data_test = pd.read_csv('/content/mask_lm_3Dec_7Dec_reindex_random_test_v3.csv', encoding='latin-1')
data_test_fixed = pd.read_csv('/content/mask_lm_3Dec_7Dec_reindex_fixed_test_v3.csv', encoding='latin-1')



from tensorflow import keras
import os
import re

# Load all files from a directory in a DataFrame.
def load_directory_data(df):
#   print('df length>>',len(df['DOCUMENT']))
  data = {}
  data["sentence"] = df['DOCUMENT']
  data["label"] =df["LABEL"]
#   print('data length>>',data)
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(df,index = None):
  df_new = load_directory_data(df[:index])
#   print(df_new)
  true_df = df_new[df_new['label'] == True]
  false_df = df_new[df_new['label'] == False]
#   print('true_df>>>',len(true_df))
#   true_df["polarity"] = 1
#   false_df["polarity"] = 0
  return pd.concat([true_df, false_df]).sample(frac=1).reset_index(drop=True)


train = load_dataset(data_train)
test = load_dataset(data_test)
dev = load_dataset(data_dev)
test_fixed = load_dataset(data_test_fixed)
print('train set length: %d, test set lenght: %d, dev set length: %d, fixed test: %d'%(len(train),len(test),len(dev),len(test_fixed)))


train set length: 22284, test set lenght: 4024, dev set length: 3480, fixed test: 5039


In [0]:
DATA_COLUMN = 'sentence'
LABEL_COLUMN = 'label'
label_list = [0,1]

In [0]:
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

dev_InputExamples = dev.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)


test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples_fixed = test_fixed.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)



## These two lines should be activated if ** is not activated
num_train_steps = int(len(train_InputExamples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Setup TPU related config
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8
ITERATIONS_PER_LOOP = 100 # I don't know what it is doing just decrease it to smaller value
# ITERATIONS_PER_LOOP = int(len(train_InputExamples) / TRAIN_BATCH_SIZE) ## set as the number of iterations in each epoch 

def get_run_config(output_dir):
  return tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=output_dir,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))



In [0]:
def create_model(is_training, input_ids, input_mask, segment_ids, labels,
                 num_labels, bert_hub_module_handle):
  """Creates a classification model."""
  tags = set()
  if is_training:
    tags.add("train")
  bert_module = hub.Module(bert_hub_module_handle, tags=tags, trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # In the demo, we are doing a simple classification task on the entire
  # segment.
  #
  # If you want to use the token-level output, use
  # bert_outputs["sequence_output"] instead.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):
    if is_training:
      # I.e., 0.1 dropout
      output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    probabilities = tf.nn.softmax(logits, axis=-1)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)

    return (loss, per_example_loss, logits, probabilities)


def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps, use_tpu, bert_hub_module_handle):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    tf.logging.info("*** Features ***")
    for name in sorted(features.keys()):
      tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    (total_loss, per_example_loss, logits, probabilities) = create_model(
        is_training, input_ids, input_mask, segment_ids, label_ids, num_labels,
        bert_hub_module_handle)

    output_spec = None
    if mode == tf.estimator.ModeKeys.TRAIN:
      train_op = optimization.create_optimizer(
          total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          train_op=train_op)
    elif mode == tf.estimator.ModeKeys.EVAL:

      def metric_fn(per_example_loss, label_ids, logits):
        predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
        accuracy = tf.metrics.accuracy(label_ids, predictions)
        loss = tf.metrics.mean(per_example_loss)
        return {
            "eval_accuracy": accuracy,
            "eval_loss": loss,
        }

      eval_metrics = (metric_fn, [per_example_loss, label_ids, logits])
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          eval_metrics=eval_metrics)
    elif mode == tf.estimator.ModeKeys.PREDICT:
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode, predictions={"probabilities": probabilities})
    else:
      raise ValueError(
          "Only TRAIN, EVAL and PREDICT modes are supported: %s" % (mode))

    return output_spec

  return model_fn

In [25]:
# Force TF Hub writes to the GS bucket we provide.
os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR
### Activate it if ** part is not activated 
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  bert_hub_module_handle=BERT_MODEL_HUB
)

estimator_from_tfhub = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)



W0429 21:53:36.923929 140603958171520 estimator.py:1924] Estimator's model_fn (<function model_fn at 0x7fe0a1c7d758>) includes params argument, but params are not passed to Estimator.


INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.26.93.186:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe0cc52ead0>, '_model_dir': 'gs://bert_example/mask_lm/models/V2/smallBERT-docLevel-seq512', '_protocol': None, '_save_checkpoints_steps': 50, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tpu_config': TPUConfig(iterations_per_loop=100, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None), '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_cluster': <tensorflow.python.distribute.cluster_resolver.tpu_cluster_resolver.TPUClusterResolver o

I0429 21:53:36.927887 140603958171520 estimator.py:201] Using config: {'_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.26.93.186:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe0cc52ead0>, '_model_dir': 'gs://bert_example/mask_lm/models/V2/smallBERT-docLevel-seq512', '_protocol': None, '_save_checkpoints_steps': 50, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tpu_config': TPUConfig(iterations_per_loop=100, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None), '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_cluster': <tensorflow.python.distribute.cluster_resolver.t

INFO:tensorflow:_TPUContext: eval_on_tpu True


I0429 21:53:36.932535 140603958171520 tpu_context.py:202] _TPUContext: eval_on_tpu True


In [0]:
# Train the model
# tf.logging.set_verbosity(tf.logging.FATAL) #DEBUG,ERROR,FATAL,INFO,WARN
def model_train(estimator):
  # We'll set sequences to be at most 128 tokens long.
  train_features = run_classifier.convert_examples_to_features(
      train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(train_InputExamples)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  tf.logging.info("  Num steps = %d", num_train_steps)
  train_input_fn = run_classifier.input_fn_builder(
      features=train_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=True)
  md = estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))
  return md



#Evaluation and Prediction 

In [0]:
def model_eval(estimator):
  # Eval the model.
  eval_examples = dev_InputExamples#processor.get_dev_examples(TASK_DATA_DIR)
  eval_features = run_classifier.convert_examples_to_features(
      eval_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(eval_examples)))
  print('  Batch size = {}'.format(EVAL_BATCH_SIZE))

  # Eval will be slightly WRONG on the TPU because it will truncate
  # the last batch.
  eval_steps = int(len(eval_examples) / EVAL_BATCH_SIZE)
  eval_input_fn = run_classifier.input_fn_builder(
      features=eval_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=True)
  result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
  print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
  output_eval_file = os.path.join(OUTPUT_DIR, "eval_results.txt")
  with tf.gfile.GFile(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    for key in sorted(result.keys()):
      print('  {} = {}'.format(key, str(result[key])))
      writer.write("%s = %s\n" % (key, str(result[key])))
      
  return result

In [0]:
import numpy as np
from sklearn import metrics
# labels = [1,0]
def model_predict(estimator,prediction_examples):
  # Make predictions on a subset of eval examples
#   prediction_examples = processor.get_dev_examples(TASK_DATA_DIR)[:PREDICT_BATCH_SIZE]
  input_features = run_classifier.convert_examples_to_features(prediction_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=True)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities']) for sentence, prediction in zip(prediction_examples, predictions)]



In [0]:
model_train(estimator_from_tfhub)

INFO:tensorflow:Writing example 0 of 2014


I0425 02:27:23.252749 140164097501056 run_classifier.py:774] Writing example 0 of 2014


INFO:tensorflow:*** Example ***


I0425 02:27:23.270198 140164097501056 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:Loss for final step: 0.92048126.


I0425 02:33:45.927968 140164097501056 estimator.py:359] Loss for final step: 0.92048126.


INFO:tensorflow:training_loop marked as finished


I0425 02:33:45.936448 140164097501056 error_handling.py:93] training_loop marked as finished


***** Finished training at 2019-04-25 02:33:45.944146 *****


In [0]:
## Run it if you want to train for a range of epochs and see the validation error and save the prediction on than
## ** Part Name **
mds = []
evs = []
pds = []
for i in range(5,11):

  NUM_TRAIN_EPOCHS = i
  
  num_train_steps = int(len(train_InputExamples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
  num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

  model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  bert_hub_module_handle=BERT_MODEL_HUB)
  
  estimator_from_tfhub = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE)
  
  md = model_train(estimator_from_tfhub)
  ev = model_eval(estimator_from_tfhub)
  pd = model_predict(estimator_from_tfhub,dev_InputExamples)
  mds.append(md)
  evs.append(ev)
  pds.append(pd)


true_label = list(dev['label'])

for pd,ev in zip(pds,evs):
  print(ev)
  labels_val = []
  for item in pd:
    labels_val.append(np.argmax(item[1]))
  print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
  print(metrics.classification_report(y_pred=labels_val,y_true = true_label))


W0429 23:42:23.793745 140603958171520 estimator.py:1924] Estimator's model_fn (<function model_fn at 0x7fe0a1209500>) includes params argument, but params are not passed to Estimator.


INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.26.93.186:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe0a6e7ab50>, '_model_dir': 'gs://bert_example/mask_lm/models/V2/smallBERT-docLevel-seq512', '_protocol': None, '_save_checkpoints_steps': 50, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tpu_config': TPUConfig(iterations_per_loop=100, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None), '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_cluster': <tensorflow.python.distribute.cluster_resolver.tpu_cluster_resolver.TPUClusterResolver o

I0429 23:42:23.798650 140603958171520 estimator.py:201] Using config: {'_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.26.93.186:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe0a6e7ab50>, '_model_dir': 'gs://bert_example/mask_lm/models/V2/smallBERT-docLevel-seq512', '_protocol': None, '_save_checkpoints_steps': 50, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tpu_config': TPUConfig(iterations_per_loop=100, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None), '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_cluster': <tensorflow.python.distribute.cluster_resolver.t

INFO:tensorflow:_TPUContext: eval_on_tpu True


I0429 23:42:23.802546 140603958171520 tpu_context.py:202] _TPUContext: eval_on_tpu True


...

I0429 23:42:24.083024 140603958171520 run_classifier.py:466] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

I0429 23:42:24.090711 140603958171520 run_classifier.py:467] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

INFO:tensorflow:label: True (id = 1)


I0429 23:42:24.095370 140603958171520 run_classifier.py:468] label: True (id = 1)


In [0]:
predictions = model_predict(estimator_from_tfhub,train_InputExamples)

INFO:tensorflow:Writing example 0 of 2014


I0425 02:33:45.986942 140164097501056 run_classifier.py:774] Writing example 0 of 2014


INFO:tensorflow:*** Example ***


I0425 02:33:45.997251 140164097501056 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

I0425 02:34:49.745388 140164097501056 tpu_estimator.py:536] Enqueue next (1) batch(es) of data to infeed.


INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.


I0425 02:34:49.748208 140164097501056 tpu_estimator.py:540] Dequeue next (1) batch(es) of data from outfeed.


INFO:tensorflow:prediction_loop marked as finished


I0425 02:34:49.917774 140164097501056 error_handling.py:93] prediction_loop marked as finished


In [0]:
# import numpy as np
# from sklearn import metrics

# labels = ["Negative","Neutral", "Positive"]
# labels_val = []
# for item in predictions:
#   labels_val.append(labels[np.argmax(item[1])])
# true_label = list(dev['sentiment'])
# print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
# print(metrics.classification_report(y_pred=labels_val,y_true = true_label))
labels_val = []
for item in predictions:
  labels_val.append(np.argmax(item[1]))
true_label = list(train['label'])
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

[[1240    0]
 [ 774    0]]
              precision    recall  f1-score   support

       False       0.62      1.00      0.76      1240
        True       0.00      0.00      0.00       774

   micro avg       0.62      0.62      0.62      2014
   macro avg       0.31      0.50      0.38      2014
weighted avg       0.38      0.62      0.47      2014



In [0]:
predictions = model_predict(estimator_from_tfhub,test_InputExamples_fixed)
labels_val = []
for item in predictions:
  labels_val.append(labels[np.argmax(item[1])])
true_label = list(test_fixed['sentiment'])




NameError: ignored

In [0]:
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

In [0]:
# tf.logging.set_verbosity(tf.logging.DEBUG) #DEBUG,ERROR,FATAL,INFO,WARN
# predictions = model_predict(estimator_from_tfhub,test_InputExamples)
labels_val = []
for item in predictions:
  labels_val.append(labels[np.argmax(item[1])])
true_label = list(test['sentiment'])


In [0]:
# len(labels_val)
# len(test_InputExamples)
len(predictions)

In [0]:
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

In [0]:
####seq 128, Small BERT, Batchsize 32 for train, 8 for dev and test
# 1)I0423 16:57:01.507206 140076901431168 basic_session_run_hooks.py:249] loss = 0.40919852, step = 46
# 2)I0423 18:08:31.116359 140500355729280 basic_session_run_hooks.py:249] loss = 0.7756149, step = 92
# 4)Loss for final step: 0.6891643.
# 5)Loss for final step: 0.9730371
# 6)Loss for final step: 0.4775733
# 7)Loss for final step: 1.2802429.
# 8)Loss for final step: 0.509207
# 9)loss = 0.29262337, step = 414
# 10)Loss for final step: 0.47267017


# 2)***** Eval results *****
#   eval_accuracy = 0.5857143
#   eval_loss = 0.8630275
#   global_step = 92
#   loss = 0.79767215
# 3)***** Eval results *****
#   eval_accuracy = 0.5857143
#   eval_loss = 0.8630275
#   global_step = 138
#   loss = 0.79767215
# 4)***** Eval results *****
#   eval_accuracy = 0.5857143
#   eval_loss = 0.86550355
#   global_step = 184
#   loss = 0.83540887
# 5)***** Eval results *****
#   eval_accuracy = 0.5964286
#   eval_loss = 0.88053304
#   global_step = 230
#   loss = 0.91362196
# 6)***** Eval results *****
#   eval_accuracy = 0.5857143
#   eval_loss = 0.90498805
#   global_step = 276
#   loss = 1.0806552
# 7)***** Eval results *****
#   eval_accuracy = 0.56785715
#   eval_loss = 0.92742974
#   global_step = 322
#   loss = 1.0180835
# 8)***** Eval results *****
#   eval_accuracy = 0.5607143
#   eval_loss = 0.9436406
#   global_step = 368
#   loss = 0.9721719
# 9)***** Eval results *****
#   eval_accuracy = 0.5607143
#   eval_loss = 0.9714315
#   global_step = 414
#   loss = 0.5798999
# 10)***** Eval results *****
#   eval_accuracy = 0.54285717
#   eval_loss = 1.0072392
#   global_step = 460
#   loss = 1.053762
  
#   DEV Info:
# 1)[[  0   0  23]
#  [  0   0  97]
#  [  0   0 164]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.00      0.00      0.00        97
#     Positive       0.58      1.00      0.73       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.19      0.33      0.24       284
# weighted avg       0.33      0.58      0.42       284
# 2)[[  0   9  14]
#  [  0  25  72]
#  [  0  21 143]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.45      0.26      0.33        97
#     Positive       0.62      0.87      0.73       164

#    micro avg       0.59      0.59      0.59       284
#    macro avg       0.36      0.38      0.35       284
# weighted avg       0.52      0.59      0.53       284
# 3)[[  0   9  14]
#  [  0  25  72]
#  [  0  21 143]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.45      0.26      0.33        97
#     Positive       0.62      0.87      0.73       164

#    micro avg       0.59      0.59      0.59       284
#    macro avg       0.36      0.38      0.35       284
# weighted avg       0.52      0.59      0.53       284
# 4)[[  0   8  42]
#  [  0  16 100]
#  [  0  17 157]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        50
#      Neutral       0.39      0.14      0.20       116
#     Positive       0.53      0.90      0.66       174

#    micro avg       0.51      0.51      0.51       340
#    macro avg       0.31      0.35      0.29       340
# weighted avg       0.40      0.51      0.41       340
# 5)[[  0  17   6]
#  [  0  40  57]
#  [  0  35 129]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.43      0.41      0.42        97
#     Positive       0.67      0.79      0.72       164

#    micro avg       0.60      0.60      0.60       284
#    macro avg       0.37      0.40      0.38       284
# weighted avg       0.54      0.60      0.56       284
# 6)[[  0  19   4]
#  [  0  44  53]
#  [  0  42 122]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.42      0.45      0.44        97
#     Positive       0.68      0.74      0.71       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.37      0.40      0.38       284
# weighted avg       0.54      0.58      0.56       284
# 7)[[  0  18   5]
#  [  0  46  51]
#  [  0  47 117]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.41      0.47      0.44        97
#     Positive       0.68      0.71      0.69       164

#    micro avg       0.57      0.57      0.57       284
#    macro avg       0.36      0.40      0.38       284
# weighted avg       0.53      0.57      0.55       284
# 8)[[  0  18   5]
#  [  0  48  49]
#  [  0  53 111]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.40      0.49      0.44        97
#     Positive       0.67      0.68      0.67       164

#    micro avg       0.56      0.56      0.56       284
#    macro avg       0.36      0.39      0.37       284
# weighted avg       0.53      0.56      0.54       284
# 9)[[  0  18   5]
#  [  0  48  49]
#  [  0  54 110]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.40      0.49      0.44        97
#     Positive       0.67      0.67      0.67       164

#    micro avg       0.56      0.56      0.56       284
#    macro avg       0.36      0.39      0.37       284
# weighted avg       0.52      0.56      0.54       284
# 10)[[  0  18   5]
#  [  0  51  46]
#  [  0  62 102]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.39      0.53      0.45        97
#     Positive       0.67      0.62      0.64       164

#    micro avg       0.54      0.54      0.54       284
#    macro avg       0.35      0.38      0.36       284
# weighted avg       0.52      0.54      0.52       284


In [0]:
###### Seq:256, small BERT, batch size 32 for train, 8 for test and dev
# 3) {'loss': 0.79650426, 'eval_accuracy': 0.58214283, 'eval_loss': 0.86125755, 'global_step': 138}
# [[  0   0  23]
#  [  0   0  97]
#  [  0   0 164]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.00      0.00      0.00        97
#     Positive       0.58      1.00      0.73       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.19      0.33      0.24       284
# weighted avg       0.33      0.58      0.42       284

# 4) {'loss': 0.73700047, 'eval_accuracy': 0.5928571, 'eval_loss': 0.8223035, 'global_step': 184}
# [[  0  11  12]
#  [  0  20  77]
#  [  0  17 147]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.42      0.21      0.28        97
#     Positive       0.62      0.90      0.73       164

#    micro avg       0.59      0.59      0.59       284
#    macro avg       0.35      0.37      0.34       284
# weighted avg       0.50      0.59      0.52       284

# 5) {'loss': 0.71594626, 'eval_accuracy': 0.5928571, 'eval_loss': 0.82239425, 'global_step': 230}
# [[  0  19   4]
#  [  0  51  46]
#  [  0  48 116]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.43      0.53      0.47        97
#     Positive       0.70      0.71      0.70       164

#    micro avg       0.59      0.59      0.59       284
#    macro avg       0.38      0.41      0.39       284
# weighted avg       0.55      0.59      0.57       284

# 6) {'loss': 0.7227276, 'eval_accuracy': 0.5857143, 'eval_loss': 0.8479867, 'global_step': 276}
# [[  0  21   2]
#  [  2  51  44]
#  [  1  49 114]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.42      0.53      0.47        97
#     Positive       0.71      0.70      0.70       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.38      0.41      0.39       284
# weighted avg       0.56      0.58      0.57       284

# 7){'loss': 0.83304703, 'eval_accuracy': 0.5642857, 'eval_loss': 0.90589315, 'global_step': 322}
# [[  0  19   4]
#  [  4  40  53]
#  [  3  42 119]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.40      0.41      0.40        97
#     Positive       0.68      0.73      0.70       164

#    micro avg       0.56      0.56      0.56       284
#    macro avg       0.36      0.38      0.37       284
# weighted avg       0.53      0.56      0.54       284

# 8){'loss': 0.84049994, 'eval_accuracy': 0.575, 'eval_loss': 0.93640614, 'global_step': 368}
# [[  2  18   3]
#  [  5  42  50]
#  [  2  44 118]]
#               precision    recall  f1-score   support

#     Negative       0.22      0.09      0.12        23
#      Neutral       0.40      0.43      0.42        97
#     Positive       0.69      0.72      0.70       164

#    micro avg       0.57      0.57      0.57       284
#    macro avg       0.44      0.41      0.42       284
# weighted avg       0.55      0.57      0.56       284

# 9){'loss': 0.8601472, 'eval_accuracy': 0.5642857, 'eval_loss': 0.95250976, 'global_step': 414}
# [[  0  18   5]
#  [  4  37  56]
#  [  1  41 122]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.39      0.38      0.38        97
#     Positive       0.67      0.74      0.70       164

#    micro avg       0.56      0.56      0.56       284
#    macro avg       0.35      0.38      0.36       284
# weighted avg       0.52      0.56      0.54       284


# 10){'loss': 0.9091368, 'eval_accuracy': 0.5535714, 'eval_loss': 0.9715489, 'global_step': 460}
# [[  0  19   4]
#  [  4  37  56]
#  [  1  44 119]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.37      0.38      0.38        97
#     Positive       0.66      0.73      0.69       164

#    micro avg       0.55      0.55      0.55       284
#    macro avg       0.34      0.37      0.36       284
# weighted avg       0.51      0.55      0.53       284

# 11){'loss': 0.9756337, 'eval_accuracy': 0.5714286, 'eval_loss': 1.0129306, 'global_step': 506}
# [[  3  16   4]
#  [  5  36  56]
#  [  1  41 122]]
#               precision    recall  f1-score   support

#     Negative       0.33      0.13      0.19        23
#      Neutral       0.39      0.37      0.38        97
#     Positive       0.67      0.74      0.71       164

#    micro avg       0.57      0.57      0.57       284
#    macro avg       0.46      0.42      0.42       284
# weighted avg       0.55      0.57      0.55       284

# 12){'loss': 0.9551747, 'eval_accuracy': 0.56785715, 'eval_loss': 1.0222387, 'global_step': 552}
# [[  3  17   3]
#  [  5  38  54]
#  [  2  43 119]]
#               precision    recall  f1-score   support

#     Negative       0.30      0.13      0.18        23
#      Neutral       0.39      0.39      0.39        97
#     Positive       0.68      0.73      0.70       164

#    micro avg       0.56      0.56      0.56       284
#    macro avg       0.45      0.42      0.42       284
# weighted avg       0.55      0.56      0.55       284

# 13){'loss': 1.0276382, 'eval_accuracy': 0.5714286, 'eval_loss': 1.0547612, 'global_step': 598}
# [[  3  17   3]
#  [  5  36  56]
#  [  1  41 122]]
#               precision    recall  f1-score   support

#     Negative       0.33      0.13      0.19        23
#      Neutral       0.38      0.37      0.38        97
#     Positive       0.67      0.74      0.71       164

#    micro avg       0.57      0.57      0.57       284
#    macro avg       0.46      0.42      0.42       284
# weighted avg       0.55      0.57      0.55       284

# 14){'loss': 1.0036505, 'eval_accuracy': 0.55714285, 'eval_loss': 1.0697843, 'global_step': 644}
# [[  3  17   3]
#  [  7  36  54]
#  [  2  44 118]]
#               precision    recall  f1-score   support

#     Negative       0.25      0.13      0.17        23
#      Neutral       0.37      0.37      0.37        97
#     Positive       0.67      0.72      0.70       164

#    micro avg       0.55      0.55      0.55       284
#    macro avg       0.43      0.41      0.41       284
# weighted avg       0.54      0.55      0.54       284

# 15){'loss': 1.0295725, 'eval_accuracy': 0.5642857, 'eval_loss': 1.0809377, 'global_step': 690}
# [[  3  17   3]
#  [  5  39  53]
#  [  1  46 117]]
#               precision    recall  f1-score   support

#     Negative       0.33      0.13      0.19        23
#      Neutral       0.38      0.40      0.39        97
#     Positive       0.68      0.71      0.69       164

#    micro avg       0.56      0.56      0.56       284
#    macro avg       0.46      0.42      0.42       284
# weighted avg       0.55      0.56      0.55       284

# 16){'loss': 1.0889318, 'eval_accuracy': 0.54642856, 'eval_loss': 1.1056138, 'global_step': 736}
# [[  3  17   3]
#  [  8  33  56]
#  [  2  44 118]]
#               precision    recall  f1-score   support

#     Negative       0.23      0.13      0.17        23
#      Neutral       0.35      0.34      0.35        97
#     Positive       0.67      0.72      0.69       164

#    micro avg       0.54      0.54      0.54       284
#    macro avg       0.42      0.40      0.40       284
# weighted avg       0.52      0.54      0.53       284

# 17){'loss': 1.1312778, 'eval_accuracy': 0.54642856, 'eval_loss': 1.1440269, 'global_step': 782}
# [[  3  17   3]
#  [  9  33  55]
#  [  2  44 118]]
#               precision    recall  f1-score   support

#     Negative       0.21      0.13      0.16        23
#      Neutral       0.35      0.34      0.35        97
#     Positive       0.67      0.72      0.69       164

#    micro avg       0.54      0.54      0.54       284
#    macro avg       0.41      0.40      0.40       284
# weighted avg       0.52      0.54      0.53       284

# 18){'loss': 1.1436831, 'eval_accuracy': 0.55, 'eval_loss': 1.1613237, 'global_step': 828}
# [[  3  16   4]
#  [  8  35  54]
#  [  2  45 117]]
#               precision    recall  f1-score   support

#     Negative       0.23      0.13      0.17        23
#      Neutral       0.36      0.36      0.36        97
#     Positive       0.67      0.71      0.69       164

#    micro avg       0.55      0.55      0.55       284
#    macro avg       0.42      0.40      0.41       284
# weighted avg       0.53      0.55      0.54       284

# 19){'loss': 1.1817628, 'eval_accuracy': 0.55, 'eval_loss': 1.1834545, 'global_step': 874}
# [[  3  16   4]
#  [  8  34  55]
#  [  1  45 118]]
#               precision    recall  f1-score   support

#     Negative       0.25      0.13      0.17        23
#      Neutral       0.36      0.35      0.35        97
#     Positive       0.67      0.72      0.69       164

#    micro avg       0.55      0.55      0.55       284
#    macro avg       0.42      0.40      0.41       284
# weighted avg       0.53      0.55      0.53       284

# 20){'loss': 1.1921012, 'eval_accuracy': 0.54642856, 'eval_loss': 1.1976833, 'global_step': 920}
# [[  3  16   4]
#  [  9  35  53]
#  [  2  47 115]]
#               precision    recall  f1-score   support

#     Negative       0.21      0.13      0.16        23
#      Neutral       0.36      0.36      0.36        97
#     Positive       0.67      0.70      0.68       164

#    micro avg       0.54      0.54      0.54       284
#    macro avg       0.41      0.40      0.40       284
# weighted avg       0.53      0.54      0.53       284

# 21){'loss': 1.2514663, 'eval_accuracy': 0.53571427, 'eval_loss': 1.2244278, 'global_step': 966}
# [[  3  16   4]
#  [ 11  32  54]
#  [  2  47 115]]
#               precision    recall  f1-score   support

#     Negative       0.19      0.13      0.15        23
#      Neutral       0.34      0.33      0.33        97
#     Positive       0.66      0.70      0.68       164

#    micro avg       0.53      0.53      0.53       284
#    macro avg       0.40      0.39      0.39       284
# weighted avg       0.51      0.53      0.52       284

# 22){'loss': 1.2630298, 'eval_accuracy': 0.5321429, 'eval_loss': 1.2527977, 'global_step': 1012}
# [[  5  14   4]
#  [ 14  32  51]
#  [  9  43 112]]
#               precision    recall  f1-score   support

#     Negative       0.18      0.22      0.20        23
#      Neutral       0.36      0.33      0.34        97
#     Positive       0.67      0.68      0.68       164

#    micro avg       0.52      0.52      0.52       284
#    macro avg       0.40      0.41      0.41       284
# weighted avg       0.52      0.52      0.52       284

# 23){'loss': 1.3330169, 'eval_accuracy': 0.53571427, 'eval_loss': 1.2815608, 'global_step': 1058}
# [[  6  13   4]
#  [ 14  31  52]
#  [  9  42 113]]
#               precision    recall  f1-score   support

#     Negative       0.21      0.26      0.23        23
#      Neutral       0.36      0.32      0.34        97
#     Positive       0.67      0.69      0.68       164

#    micro avg       0.53      0.53      0.53       284
#    macro avg       0.41      0.42      0.42       284
# weighted avg       0.53      0.53      0.53       284

# 24){'loss': 1.3111317, 'eval_accuracy': 0.5321429, 'eval_loss': 1.2898273, 'global_step': 1104}
# [[  5  14   4]
#  [ 14  32  51]
#  [  9  43 112]]
#               precision    recall  f1-score   support

#     Negative       0.18      0.22      0.20        23
#      Neutral       0.36      0.33      0.34        97
#     Positive       0.67      0.68      0.68       164

#    micro avg       0.52      0.52      0.52       284
#    macro avg       0.40      0.41      0.41       284
# weighted avg       0.52      0.52      0.52       284

In [0]:
## Larg BERT seq:256, batchsize 8 for train, dev , test
# {'loss': 1.0679293, 'eval_accuracy': 0.5955882, 'eval_loss': 0.8965841, 'global_step': 93}
# [[  0   0  23]
#  [  0   0  97]
#  [  0   0 164]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.00      0.00      0.00        97
#     Positive       0.58      1.00      0.73       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.19      0.33      0.24       284
# weighted avg       0.33      0.58      0.42       284

# {'loss': 1.0362501, 'eval_accuracy': 0.5955882, 'eval_loss': 0.88540924, 'global_step': 186}
# [[  0   0  23]
#  [  0   0  97]
#  [  0   0 164]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.00      0.00      0.00        97
#     Positive       0.58      1.00      0.73       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.19      0.33      0.24       284
# weighted avg       0.33      0.58      0.42       284

# {'loss': 1.0290864, 'eval_accuracy': 0.5955882, 'eval_loss': 0.8812368, 'global_step': 279}
# [[  0   0  23]
#  [  0   0  97]
#  [  0   0 164]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.00      0.00      0.00        97
#     Positive       0.58      1.00      0.73       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.19      0.33      0.24       284
# weighted avg       0.33      0.58      0.42       284

# {'loss': 1.0271251, 'eval_accuracy': 0.5955882, 'eval_loss': 0.88323754, 'global_step': 372}
# [[  0   0  23]
#  [  0   0  97]
#  [  0   0 164]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.00      0.00      0.00        97
#     Positive       0.58      1.00      0.73       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.19      0.33      0.24       284
# weighted avg       0.33      0.58      0.42       284

# {'loss': 1.02654, 'eval_accuracy': 0.5955882, 'eval_loss': 0.8804489, 'global_step': 465}
# [[  0   0  23]
#  [  0   0  97]
#  [  0   0 164]]
#               precision    recall  f1-score   support

#     Negative       0.00      0.00      0.00        23
#      Neutral       0.00      0.00      0.00        97
#     Positive       0.58      1.00      0.73       164

#    micro avg       0.58      0.58      0.58       284
#    macro avg       0.19      0.33      0.24       284
# weighted avg       0.33      0.58      0.42       284

# SAME FOR 10 EPOCHS!!!!!

In [0]:
predictions